In [ ]:
import cv2
import mediapipe as mp
import math

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Start webcam capture
cap = cv2.VideoCapture(0)

def distance(point1, point2):
    """Calculate the Euclidean distance between two points."""
    return math.sqrt((point1.x - point2.x) ** 2 + (point1.y - point2.y) ** 2)

def is_peace_sign(landmarks):
    """Check if the hand is making a peace sign (✌️)."""
    index_finger_up = landmarks[8].y < landmarks[6].y
    middle_finger_up = landmarks[12].y < landmarks[10].y
    
    ring_finger_down = landmarks[16].y > landmarks[14].y
    pinky_finger_down = landmarks[20].y > landmarks[18].y

    return index_finger_up and middle_finger_up and ring_finger_down and pinky_finger_down

def is_ok_sign(landmarks):
    """Check if the hand is making an OK sign (👌)."""
    middle_finger_up = landmarks[12].y < landmarks[10].y
    ring_finger_up = landmarks[16].y < landmarks[14].y
    pinky_finger_up = landmarks[20].y < landmarks[18].y

    thumb_index_distance = distance(landmarks[4], landmarks[8])  # Thumb touching index fingertip
    index_finger_curled = landmarks[8].y > landmarks[6].y  # Index finger is curled

    return middle_finger_up and ring_finger_up and pinky_finger_up and index_finger_curled and thumb_index_distance < 0.05

def is_rock_n_roll(landmarks):
    """Check if the hand is making the Rock 'n' Roll sign (🤘)."""
    index_finger_up = landmarks[8].y < landmarks[6].y
    pinky_finger_up = landmarks[20].y < landmarks[18].y

    middle_finger_down = landmarks[12].y > landmarks[10].y
    ring_finger_down = landmarks[16].y > landmarks[14].y

    return index_finger_up and pinky_finger_up and middle_finger_down and ring_finger_down

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB (MediaPipe uses RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Check for gestures
            if is_peace_sign(hand_landmarks.landmark):
                #print("✌️ Peace!")
                
                #call follow mode <--
                
            elif is_ok_sign(hand_landmarks.landmark):
                #print("👌 OK!")
                
                #exit follow mode <--
                
            elif is_rock_n_roll(hand_landmarks.landmark):
                #print("🤘 Rock 'n' Roll!")
                
                #exit follow mode <--
                
                cap.release()
                cv2.destroyAllWindows()

    cv2.imshow("Gesture Detection", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()